In [11]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
import logging
import datetime

from sklearn.preprocessing import OneHotEncoder,MinMaxScaler, Normalizer, LabelEncoder
from sklearn.feature_selection import SelectKBest,chi2,SelectFromModel
from xgboost import XGBClassifier, XGBRegressor
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import GridSearchCV
from sklearn.impute import SimpleImputer
from xgboost import plot_importance
from mlxtend.preprocessing import DenseTransformer
from mlxtend.feature_selection import ColumnSelector
from itertools import product
import gc

sns.set(color_codes=True)

In [2]:
def _transfer_type(df, cols, dtype):
    for v in cols:
        df[v] = df[v].astype(dtype)
    
    return df

# Features Engineering

## outliers

从外部获取所有数据集合，做类型整理，方便后面进行使用，使得pandas的dataframe所占用的内存变小

In [3]:
dateparse = lambda x: pd.datetime.strptime(x, '%d.%m.%Y')
sales_df = pd.read_csv('../input/sales_train.csv', parse_dates = ['date'], date_parser=dateparse)

item_df = pd.read_csv('../input/items.csv')
shop_df = pd.read_csv('../input/shops.csv')
category_df = pd.read_csv('../input/item_categories.csv')

test_df = pd.read_csv('../input/test.csv').set_index('ID')

数据类型整理，减少空间使用

In [4]:
sales_df = _transfer_type(sales_df, ['date_block_num', 'shop_id', 'item_id', 'item_cnt_day'], 'int16')
sales_df = _transfer_type(sales_df, ['item_price'], 'float32')

item_df = _transfer_type(item_df, ['item_id', 'item_category_id'], 'int16')
item_df['item_name'] = item_df['item_name'].astype('str')

shop_df['shop_name'] = shop_df['shop_name'].astype('str')
shop_df['shop_id'] = shop_df['shop_id'].astype('int16')

category_df['item_category_name'] = category_df['item_category_name'].astype('str')
category_df['item_category_id'] = category_df['item_category_id'].astype('int16')

### Clear the data out of range

In [5]:
sales_df = sales_df[sales_df.item_price<100000]
sales_df = sales_df[sales_df.item_cnt_day<1000]

### Correct the negative item price

In [6]:
sales_df[sales_df.item_price < 0]

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
484683,2013-05-15,4,32,2973,-1.0,1


In [7]:
sales_df.at[484683, 'item_price'] = sales_df[(sales_df.item_id == 2973) & (sales_df.item_price > 0)].item_price.mean()

### Adjust duplicates by shop name

According to the reference, some shops are duplicates. This feature found is based on the language background.

In [8]:
shop_df[shop_df.shop_id == 0]

,shop_name,shop_id
0,"!Якутск Орджоникидзе, 56 фран",0


In [9]:
shop_df[shop_df.shop_id == 57]

,shop_name,shop_id
57,"Якутск Орджоникидзе, 56",57


The shop_id 0 and 57 are the same shop. And other pairs are 1 and 58, 10 and 11.

In [10]:
# Якутск Орджоникидзе, 56
sales_df.at[sales_df.shop_id == 0, 'shop_id'] = 57
# Якутск ТЦ "Центральный"
sales_df.at[sales_df.shop_id == 1, 'shop_id'] = 58
# Жуковский ул. Чкалова 39м²
sales_df.at[sales_df.shop_id == 10, 'shop_id'] = 11

## Extract Basic Features

+ **Feature 'r' series**: Sales record

In [12]:
def _rename(prefix):
    cols = ['2013-01', '2013-02', '2013-03', '2013-04','2013-05','2013-06','2013-07','2013-08','2013-09','2013-10','2013-11','2013-12','2014-01','2014-02','2014-03','2014-04','2014-05','2014-06','2014-07','2014-08','2014-09','2014-10','2014-11','2014-12','2015-01','2015-02','2015-03','2015-04','2015-05','2015-06','2015-07','2015-08', '2015-09','2015-10']
    
    result = {}
    for i in range(1, len(cols) + 1, 1):
        result[cols[i-1]] = prefix + str(i)
    
    return result

In [13]:
t = sales_df.groupby([sales_df.date.apply(lambda x: x.strftime('%Y-%m')),'item_id','shop_id']).agg({'item_cnt_day': 'sum'}).reset_index()
t = t[['date','item_id','shop_id','item_cnt_day']]
t = t.pivot_table(index=['item_id','shop_id'], columns='date',values='item_cnt_day',fill_value=0).reset_index()
t = t.rename(index=str, columns=_rename('r'))
sales_record_df = t.copy()

del t
gc.collect()


for key in sales_record_df.columns.values:
    sales_record_df[key] = sales_record_df[key].astype('int16')

+ **Feature 'city'**: Each shop_name starts with the city name.

In [14]:
shop_df.loc[shop_df.shop_name == 'Сергиев Посад ТЦ "7Я"', 'shop_name'] = 'СергиевПосад ТЦ "7Я"'
shop_df['shop_name'] = shop_df['shop_name'].astype(str)
shop_df['city'] = shop_df['shop_name'].str.split(' ').map(lambda x: x[0])
shop_df.loc[shop_df.city == '!Якутск', 'city'] = 'Якутск'

encoder = LabelEncoder()
encoder.fit(shop_df['city'])
shop_df['city_code'] = encoder.transform(shop_df['city'])

+ **Feature 'type'**: Each category contains type and subtype in its name.
+ **Feature 'subtype'**: Each category contains type and subtype in its name.

In [15]:
category_df['item_category_name'] = category_df['item_category_name'].astype(str)
category_df['type'] = category_df['item_category_name'].map(lambda x: x.split('-')[0].strip())

encoder = LabelEncoder()
encoder.fit(category_df['type'])
category_df['type_code'] = encoder.transform(category_df['type'])

In [16]:
category_df['subtype'] = category_df['item_category_name'].map(lambda x: x.split('-')[1].strip() if len(x.split('-')) > 1 else x.split('-')[0].strip())

encoder = LabelEncoder()
encoder.fit(category_df['subtype'])
category_df['subtype_code'] = encoder.transform(category_df['subtype'])

## Merge Data into one entity

In [17]:
sales_record_df = pd.merge(sales_record_df, item_df, on=['item_id'], how='left')
sales_record_df = pd.merge(sales_record_df, shop_df, on=['shop_id'], how='left')
sales_record_df = pd.merge(sales_record_df, category_df, on=['item_category_id'], how='left')

sales_record_df = sales_record_df.drop(['city', 'type', 'subtype', 'item_name', 'shop_name', 'item_category_name'], axis=1)
sales_record_df = _transfer_type(sales_record_df, ['item_category_id', 'city_code', 'type_code', 'subtype_code'], 'int16')

sales_record_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 418908 entries, 0 to 418907
Data columns (total 40 columns):
item_id             418908 non-null int16
shop_id             418908 non-null int16
r1                  418908 non-null int16
r2                  418908 non-null int16
r3                  418908 non-null int16
r4                  418908 non-null int16
r5                  418908 non-null int16
r6                  418908 non-null int16
r7                  418908 non-null int16
r8                  418908 non-null int16
r9                  418908 non-null int16
r10                 418908 non-null int16
r11                 418908 non-null int16
r12                 418908 non-null int16
r13                 418908 non-null int16
r14                 418908 non-null int16
r15                 418908 non-null int16
r16                 418908 non-null int16
r17                 418908 non-null int16
r18                 418908 non-null int16
r19                 418908 non-null int16
r20    

## Extract Aggragated Features

In [28]:
for i in range(1, 35, 1):
    t = sales_record_df.groupby(['item_category_id', 'shop_id'])['r'+str(i)].mean().astype('float32').reset_index(name='si'+str(i))
    sales_record_df = pd.merge(sales_record_df, t, on=['item_category_id', 'shop_id'], how='left')

In [29]:
for i in range(1, 35, 1):
    t = sales_record_df.groupby(['item_id'])['r'+str(i)].mean().astype('float32').reset_index(name='i'+str(i))
    sales_record_df = pd.merge(sales_record_df, t, on=['item_id'], how='left')

In [30]:
for i in range(1, 35, 1):
    t = sales_record_df.groupby(['shop_id'])['r'+str(i)].mean().astype('float32').reset_index(name='s'+str(i))
    sales_record_df = pd.merge(sales_record_df, t, on=['shop_id'], how='left')

In [31]:
for i in range(1, 35, 1):
    t = sales_record_df.groupby(['item_category_id'])['r'+str(i)].mean().astype('float32').reset_index(name='ic'+str(i))
    sales_record_df = pd.merge(sales_record_df, t, on=['item_category_id'], how='left')

In [32]:
sales_record_df = sales_record_df.rename(index=str, columns={'r34': 'label'})
sales_record_df = sales_record_df.drop(['si34', 'i34', 's34', 'ic34'], axis=1)

In [19]:
def _mean(df, prefix):
    arr = []
    for i in range(1, 34, 1):
        arr.append(prefix+str(i))
    
    df[prefix+'_m'] = df[arr].mean()
    df[prefix+'_m'] = df[prefix+'_m'].astype('float32')
    return df

In [20]:
_mean(sales_record_df, 'r')

,item_id,shop_id,r1,r2,r3,r4,r5,r6,r7,r8,...,r30,r31,r32,r33,r34,item_category_id,city_code,type_code,subtype_code,r_m
0,0,54,0,0,0,0,0,0,0,0,...,0,0,0,0,0,40,26,11,4,NaN
1,1,55,0,0,0,0,0,0,0,0,...,0,0,0,0,0,76,27,15,31,NaN
2,2,54,0,0,0,0,0,0,0,0,...,0,0,0,0,0,40,26,11,4,NaN
3,3,54,0,0,0,0,0,0,0,0,...,0,0,0,0,0,40,26,11,4,NaN
4,4,54,0,0,0,0,0,0,0,0,...,0,0,0,0,0,40,26,11,4,NaN
5,5,54,0,0,0,0,0,0,0,0,...,0,0,0,0,0,40,26,11,4,NaN
6,6,54,0,0,0,0,0,0,0,0,...,0,0,0,0,0,40,26,11,4,NaN
7,7,54,0,0,0,0,0,0,0,0,...,0,0,0,0,0,40,26,11,4,NaN
8,8,54,0,0,0,0,0,0,0,0,...,0,0,0,0,0,40,26,11,4,NaN
9,9,54,0,0,0,0,0,0,0,0,...,0,0,0,0,0,40,26,11,4,NaN


In [36]:
sales_record_df = sales_record_df.fillna(0)
sales_record_df = _mean(sales_record_df, 'r')
sales_record_df = _mean(sales_record_df, 'si')
sales_record_df = _mean(sales_record_df, 'i')
sales_record_df = _mean(sales_record_df, 's')
sales_record_df = _mean(sales_record_df, 'ic')

In [37]:
sales_record_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 418908 entries, 0 to 418907
Columns: 177 entries, item_id to ic_m
dtypes: float32(137), int16(40)
memory usage: 254.1+ MB


In [38]:
sales_record_df.columns.values

array(['item_id', 'shop_id', 'r1', 'r2', 'r3', 'r4', 'r5', 'r6', 'r7',
       'r8', 'r9', 'r10', 'r11', 'r12', 'r13', 'r14', 'r15', 'r16', 'r17',
       'r18', 'r19', 'r20', 'r21', 'r22', 'r23', 'r24', 'r25', 'r26',
       'r27', 'r28', 'r29', 'r30', 'r31', 'r32', 'r33', 'label',
       'item_category_id', 'city_code', 'type_code', 'subtype_code',
       'si1', 'si2', 'si3', 'si4', 'si5', 'si6', 'si7', 'si8', 'si9',
       'si10', 'si11', 'si12', 'si13', 'si14', 'si15', 'si16', 'si17',
       'si18', 'si19', 'si20', 'si21', 'si22', 'si23', 'si24', 'si25',
       'si26', 'si27', 'si28', 'si29', 'si30', 'si31', 'si32', 'si33',
       'i1', 'i2', 'i3', 'i4', 'i5', 'i6', 'i7', 'i8', 'i9', 'i10', 'i11',
       'i12', 'i13', 'i14', 'i15', 'i16', 'i17', 'i18', 'i19', 'i20',
       'i21', 'i22', 'i23', 'i24', 'i25', 'i26', 'i27', 'i28', 'i29',
       'i30', 'i31', 'i32', 'i33', 's1', 's2', 's3', 's4', 's5', 's6',
       's7', 's8', 's9', 's10', 's11', 's12', 's13', 's14', 's15', 's16',
       

In [39]:
sales_record_df.to_csv('../features/train.csv', index=False)

## Feature Selection

## Train

In [3]:
dataset = pd.read_csv('../features/train.csv')

In [4]:
dataset = dataset.fillna(0)

In [5]:
features = ['item_id', 'shop_id', 'r1', 'r2', 'r3', 'r4', 'r5', 'r6', 'r7',
       'r8', 'r9', 'r10', 'r11', 'r12', 'r13', 'r14', 'r15', 'r16', 'r17',
       'r18', 'r19', 'r20', 'r21', 'r22', 'r23', 'r24', 'r25', 'r26',
       'r27', 'r28', 'r29', 'r30', 'r31', 'r32', 'r33', 
       'item_category_id', 'city_code', 'type_code', 'subtype_code',
       'si1', 'si2', 'si3', 'si4', 'si5', 'si6', 'si7', 'si8', 'si9',
       'si10', 'si11', 'si12', 'si13', 'si14', 'si15', 'si16', 'si17',
       'si18', 'si19', 'si20', 'si21', 'si22', 'si23', 'si24', 'si25',
       'si26', 'si27', 'si28', 'si29', 'si30', 'si31', 'si32', 'si33',
       'i1', 'i2', 'i3', 'i4', 'i5', 'i6', 'i7', 'i8', 'i9', 'i10', 'i11',
       'i12', 'i13', 'i14', 'i15', 'i16', 'i17', 'i18', 'i19', 'i20',
       'i21', 'i22', 'i23', 'i24', 'i25', 'i26', 'i27', 'i28', 'i29',
       'i30', 'i31', 'i32', 'i33', 's1', 's2', 's3', 's4', 's5', 's6',
       's7', 's8', 's9', 's10', 's11', 's12', 's13', 's14', 's15', 's16',
       's17', 's18', 's19', 's20', 's21', 's22', 's23', 's24', 's25',
       's26', 's27', 's28', 's29', 's30', 's31', 's32', 's33', 'ic1',
       'ic2', 'ic3', 'ic4', 'ic5', 'ic6', 'ic7', 'ic8', 'ic9', 'ic10',
       'ic11', 'ic12', 'ic13', 'ic14', 'ic15', 'ic16', 'ic17', 'ic18',
       'ic19', 'ic20', 'ic21', 'ic22', 'ic23', 'ic24', 'ic25', 'ic26',
       'ic27', 'ic28', 'ic29', 'ic30', 'ic31', 'ic32', 'ic33', 'r_m',
       'si_m', 'i_m', 's_m', 'ic_m']

train_dataset_x = dataset[features]
train_dataset_y = dataset['label']

In [6]:
import xgboost as xgb

params = {
  'booster': 'gbtree',
  'objective': 'reg:linear',
  'eta': 0.1,
  'gamma': 0.7000000000000001,
  'min_child_weight': 7,
  'max_depth': 4,
  'subsample': 0.5,
  'colsample_bytree': 0.1,
  'num_round': 190,
  'nthread': 2,
  'silent': 0,
  'seed': 2019,
  "max_evals": 200,
}

xgb_dmatrix = xgb.DMatrix(train_dataset_x, label=train_dataset_y)
model = xgb.train(params, xgb_dmatrix, num_boost_round=190)

[15:52:46] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=4
[15:52:48] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 22 extra nodes, 0 pruned nodes, max_depth=4
[15:52:50] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 22 extra nodes, 0 pruned nodes, max_depth=4
[15:52:51] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 12 extra nodes, 0 pruned nodes, max_depth=4
[15:52:53] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 20 extra nodes, 0 pruned nodes, max_depth=4
[15:52:54] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 16 extra nodes, 0 pruned nodes, max_depth=4
[15:52:56] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 12 extra nodes, 0 pruned nodes, max_depth=4
[15:52:57] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 20 extra nodes, 0 pruned nodes, max_depth=4
[15:52:59] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 18 extra nodes, 0 pruned nodes, max_

In [7]:
from sklearn.metrics import mean_squared_error
from math import sqrt

pred = model.predict(xgb_dmatrix)
sqrt(mean_squared_error(train_dataset_y, pred))

1.1359275457309692

## Predict

In [44]:
predict_dataset = pd.read_csv('../features/features.csv')
predict_dataset = predict_dataset.fillna(0)

In [45]:
test_df = pd.read_csv('../input/test.csv').set_index('ID')
test_df.shape

(214200, 2)

In [46]:
cols = ['item_id', 'shop_id', 'r1', 'r2', 'r3', 'r4', 'r5', 'r6', 'r7',
       'r8', 'r9', 'r10', 'r11', 'r12', 'r13', 'r14', 'r15', 'r16', 'r17',
       'r18', 'r19', 'r20', 'r21', 'r22', 'r23', 'r24', 'r25', 'r26',
       'r27', 'r28', 'r29', 'r30', 'r31', 'r32', 'r33', 'r34',
       'item_category_id', 'city_code', 'type_code', 'subtype_code']
dataset_predict = pd.merge(test_df, predict_dataset[cols], on=['item_id', 'shop_id'], how='left')
dataset_predict = dataset_predict.fillna(0)

从这里的结果来看，如何预测在前面集合中没有出现过的[shop_id, item_id]是最为关键的

In [47]:
def _rename(df, prefix, start, end):
    arr = []
    for i in range(0, 34, 1):
        arr.append(prefix + str(i))

    rnames = dict(zip(df.columns.values[start:end], arr))
    df = df.rename(index=str, columns=rnames)
    return df

In [48]:
dataset_predict = _rename(dataset_predict, 'r', 2, 36)

In [49]:
for i in range(0, 34, 1):
    t = dataset_predict.groupby(['item_category_id', 'shop_id'])['r'+str(i)].mean().astype('float32').reset_index(name='si'+str(i))
    dataset_predict = pd.merge(dataset_predict, t, on=['item_category_id', 'shop_id'], how='left')

In [50]:
for i in range(0, 34, 1):
    t = dataset_predict.groupby(['item_id'])['r'+str(i)].mean().astype('float32').reset_index(name='i'+str(i))
    dataset_predict = pd.merge(dataset_predict, t, on=['item_id'], how='left')

In [51]:
for i in range(0, 34, 1):
    t = dataset_predict.groupby(['shop_id'])['r'+str(i)].mean().astype('float32').reset_index(name='s'+str(i))
    dataset_predict = pd.merge(dataset_predict, t, on=['shop_id'], how='left')

In [52]:
for i in range(0, 34, 1):
    t = dataset_predict.groupby(['item_category_id'])['r'+str(i)].mean().astype('float32').reset_index(name='ic'+str(i))
    dataset_predict = pd.merge(dataset_predict, t, on=['item_category_id'], how='left')

In [53]:
dataset_predict = _mean(dataset_predict, 'r')
dataset_predict = _mean(dataset_predict, 'si')
dataset_predict = _mean(dataset_predict, 'i')
dataset_predict = _mean(dataset_predict, 's')
dataset_predict = _mean(dataset_predict, 'ic')

In [54]:
dataset_predict.shape

(214200, 181)

In [55]:
predict_dmatrix = xgb.DMatrix(dataset_predict[features])
pred = model.predict(predict_dmatrix)

In [56]:
pred = list(map(lambda x: min(20,max(x,0)), list(pred)))

submission = pd.DataFrame({
    "ID": test_df.index, 
    "item_cnt_month": pred
})

submission.to_csv('submission.csv', index=False)

# Reference

## Normal ML Analysis

+ [Feature engineering, xgboost](https://www.kaggle.com/dlarionov/feature-engineering-xgboost)
+ [Predicting sales using Lightgbm](https://www.kaggle.com/sanket30/predicting-sales-using-lightgbm)

## Time Series

对于revenge的预测很有帮助，但是对于单个店的单个商品而言没有实际的意义，这种预测方式的学习非常有益处

+ [AR(I)MA时间序列建模过程——步骤和python代码](https://www.jianshu.com/p/cced6617b423)
+ [python时间序列分析](http://www.cnblogs.com/foley/p/5582358.html)
+ [AR、MA及ARMA模型](https://zhuanlan.zhihu.com/p/22248464)
+ [Time Series with Python (ODSC) STA.ipynb](https://github.com/ultimatist/ODSC17/blob/master/Time%20Series%20with%20Python%20(ODSC)%20STA.ipynb)
+ [Getting Started with Time Series](https://pyflux.readthedocs.io/en/latest/getting_started.html)
+ [Welcome to Statsmodels’s Documentation](http://www.statsmodels.org/devel/index.html)